In [2]:
%run ../../../utils/commonImports.py
%run ../../../utils/tradingImports.py
%matplotlib inline

from exputils import *

C:\Users\david.vesely\AppData\Local\conda\conda\envs\crypto36\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Load data

In [3]:
dropbox_dir = 'D:\\Dropbox\\My work\\krypl-project'
# dropbox_dir = '/Users/david.vesely/Dropbox/My work/krypl-project'
features = ['support']
pairs = read_tsv('selected-pairs.tsv', header=None)[0].tolist()

data_dict = {p: read_all(p, dropbox_dir) for p in pairs}
supports = read_tsv('supports.tsv')

# Label supports

In [15]:
window_sizes = [4]

DAY = 48
max_hold_time = 5 * DAY
stop_loss = 0.05
fee = 0.0025

In [16]:
from tqdm import tqdm_notebook

def return_with_fee(price_s, price_e, fee):
    fee_part = (1 - fee) ** 2
    return ((price_e * fee_part) / price_s) - 1


def max_profit(data, index):
    if len(data) < index + 1:
        return None
    last = index + max_hold_time if len(data) > index + max_hold_time else len(data) - 1
    selected = reset_index_hard(data.iloc[index+1:last, :].copy())
    buy_price = selected['close'].iloc[0]
    selected['_return'] = (selected['close'] / buy_price) - 1
    stop_losses = selected[selected['_return'] <= -stop_loss].reset_index()
    if len(stop_losses) > 0:
        stop_loss_i = stop_losses['index'].min()
        selected = selected.iloc[:stop_loss_i, :]
        
    return return_with_fee(buy_price, selected['high'].max(), fee)

n = len(pairs) * len(window_sizes)
with tqdm_notebook(total=n) as pbar:
    supports_data = pd.DataFrame()
    for pair in pairs:
        for win_size in window_sizes:
            data = data_dict[pair]
            supports_df = supports.query(f'pair == "{pair}"')\
                .query(f'win_size == {win_size}')
            
            supports_df['max_profit'] = [
                max_profit(data, i) for i in supports_df['index'].apply(int)
            ]
            
            supports_data = supports_data.append(supports_df)
            pbar.update()

In [17]:
write_tsv(supports_data, 'labeled.tsv')